# Customize output format in prompt

## Load model

In [1]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff4ff8e3c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff4ffac8f0>, root_client=<openai.OpenAI object at 0xffff50962cf0>, root_async_client=<openai.AsyncOpenAI object at 0xffff4ff8e420>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Load prompt

In [2]:
from langchain_core.prompts import load_prompt
prompt = load_prompt(path='../prompts/single-day/ES.json')

prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')

## Custom Output Parser

### Create Pydantic model

`/workspace/modules/models.py`

In [3]:
from modules.models import News

### Associate Pydantic model with parser

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=News)

parser

PydanticOutputParser(pydantic_object=<class 'modules.models.News'>)

### Add parser to prompt

In [5]:
prompt.template = 'Answer the QUERY in Spanish and following the format: {format_instructions}\n\nQUERY:\n\n' + prompt.template
prompt.template 

'Answer the QUERY in Spanish and following the format: {format_instructions}\n\nQUERY:\n\nEl día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.'

In [6]:
prompt.partial_variables.update({"format_instructions": parser.get_format_instructions()})
prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"stock": {"description": "símbolo de la acción", "title": "Stock", "type": "string"}, "date": {"description": "fecha de la noticia", "title": "Date", "type": "string"}, "value": {"description": "valor acumulado significativo de la acción", "title": "Value", "type": "number"}, "title": {"description": "título de la noticia", "title": "Title", "type": "string"}, "url": {"description": "url de

## Chain

### Define chain

In [7]:
chain = prompt | model | parser
chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"stock": {"description": "símbolo de la acción", "title": "Stock", "type": "string"}, "date": {"description": "fecha de la noticia", "title": "Date", "type": "string"}, "value": {"description": "valor acumulado significativo de la acción", "title": "Value", "type": "number"}, "title": {"description": "título de la noticia", "title": "Title", "type": "string"}, "url": {"description": "url de

### Preprocess input data

In [8]:
ticker = 'AAPL'
freq = 'D'

In [10]:
from modules import utils

r = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=1
)

r

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Date
2024-06-11    7.26492
Name: AAPL, dtype: float64

In [11]:
date = r.index[0].strftime('%Y-%m-%d')
change = round(r.values[0], 2)

date, change

('2024-06-11', 7.26)

### Invoke chain

In [12]:
output = chain.invoke({
    'DATE': date,
    'RETURN': change,
    'TICKER': ticker,
    'FREQUENCY': freq
})

print(output)

stock='AAPL' date='2024-06-11' value=7.26 title='La manzana dorada: Acciones de Apple se disparan y alcanzan nuevo récord tras acuerdo con OpenAI' url='https://www.elfinanciero.com.mx/bloomberg/2024/06/11/acciones-de-apple-se-disparan-y-alcalza-nuevo-record-tras-acuerdo-con-chatgpt-openai/' source='El Financiero' explanation='Las acciones de Apple subieron un 7.3% tras anunciar su asociación con OpenAI para incorporar ChatGPT en sus dispositivos, lo que generó expectativas positivas sobre la integración de la inteligencia artificial en sus productos.'


In [13]:
data = output.model_dump()
data

{'stock': 'AAPL',
 'date': '2024-06-11',
 'value': 7.26,
 'title': 'La manzana dorada: Acciones de Apple se disparan y alcanzan nuevo récord tras acuerdo con OpenAI',
 'url': 'https://www.elfinanciero.com.mx/bloomberg/2024/06/11/acciones-de-apple-se-disparan-y-alcalza-nuevo-record-tras-acuerdo-con-chatgpt-openai/',
 'source': 'El Financiero',
 'explanation': 'Las acciones de Apple subieron un 7.3% tras anunciar su asociación con OpenAI para incorporar ChatGPT en sus dispositivos, lo que generó expectativas positivas sobre la integración de la inteligencia artificial en sus productos.'}

In [14]:
import pandas as pd
pd.DataFrame([data]).style

,stock,date,value,title,url,source,explanation
0,AAPL,2024-06-11,7.260000,La manzana dorada: Acciones de Apple se disparan y alcanzan nuevo récord tras acuerdo con OpenAI,https://www.elfinanciero.com.mx/bloomberg/2024/06/11/acciones-de-apple-se-disparan-y-alcalza-nuevo-record-tras-acuerdo-con-chatgpt-openai/,El Financiero,"Las acciones de Apple subieron un 7.3% tras anunciar su asociación con OpenAI para incorporar ChatGPT en sus dispositivos, lo que generó expectativas positivas sobre la integración de la inteligencia artificial en sus productos."


## Multiple days

In [16]:
r = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=5
)

r

[*********************100%***********************]  1 of 1 completed


Date
2024-06-11    7.264920
2024-05-03    5.981614
2024-08-05   -4.816705
2024-04-11    4.327102
2024-03-21   -4.085741
Name: AAPL, dtype: float64

In [17]:
results = []

for date, change in r.items():
    try:
        output = chain.invoke({
            'DATE': date.strftime('%Y-%m-%d'),
            'RETURN': round(change, 2),
            'TICKER': ticker,
            'FREQUENCY': freq
        })
        results.append(output.model_dump())
    except Exception as e:
        print(f"Error: {e}")
        continue

Error: Invalid json output: El 5 de agosto de 2024, las acciones de Apple Inc. (AAPL) experimentaron una caída significativa del 4,82%. Este descenso se enmarca en un contexto de turbulencias en los mercados financieros globales, conocido como "lunes negro", que afectó a diversas bolsas alrededor del mundo.

El desplome fue impulsado por el temor a una recesión en Estados Unidos, exacerbado por la creación de solo 114.000 empleos en julio, cifra menor a lo esperado, y un aumento en la tasa de desempleo al 4,3%. Estos factores generaron desconfianza entre los inversores, afectando especialmente a las grandes empresas tecnológicas como Apple. ([elpais.com](https://elpais.com/expres/2024-08-06/claves-del-lunes-negro-en-bolsa-que-significa-para-la-economia-mundial.html?utm_source=openai))

Además, la caída de la bolsa japonesa, con el índice Nikkei hundiéndose un 12,4%, su mayor desplome desde 1987, contribuyó a la inestabilidad en los mercados globales. En Wall Street, el índice Nasdaq Co

In [19]:
df = pd.DataFrame([r for r in results])
df.style

,stock,date,value,title,url,source,explanation
0,AAPL,2024-06-11,7.260000,Acciones de Apple se disparan un 7% tras el anuncio de su propia Inteligencia Artificial,https://nelpr.com/2024/06/11/acciones-de-apple-se-disparan-un-7-tras-el-anuncio-de-su-propia-inteligencia-artificial/,Nelpr,"El anuncio de 'Apple Intelligence', la plataforma de inteligencia artificial de Apple, durante la WWDC 2024, generó un optimismo significativo entre los inversores, impulsando el valor de las acciones."
1,AAPL,2024-05-03,5.980000,Apple informa los resultados del segundo trimestre,https://www.apple.com/es/newsroom/2024/05/apple-reports-second-quarter-results/,Apple Newsroom,"El 2 de mayo de 2024, Apple anunció los resultados financieros del segundo trimestre fiscal, reportando ingresos de 90.800 millones de dólares y un beneficio por acción de 1,53 dólares. A pesar de una disminución del 4% en ingresos interanuales, la compañía destacó un récord histórico en ingresos por servicios y anunció un programa de recompra de acciones por 110.000 millones de dólares. Estas noticias positivas probablemente impulsaron la confianza de los inversores, resultando en un aumento significativo del 5,98% en el precio de las acciones de Apple el 3 de mayo de 2024."
2,AAPL,2024-04-11,4.330000,Apple desarrolla chips de IA para Mac,https://eltiempolatino.com/2024/04/11/economia/sp-500-y-nasdaq-cerraron-con-ganancias-impulsados-por-apple-y-nvidia/,El Tiempo Latino,"El informe sobre el desarrollo de chips de inteligencia artificial para Mac por parte de Apple generó optimismo entre los inversores, impulsando el precio de las acciones."
3,AAPL,2024-03-21,-4.090000,El Departamento de Justicia de Estados Unidos demanda a Apple por prácticas antimonopolio,https://as.com/us/actualidad/por-que-el-departamento-de-justicia-de-estados-unidos-demando-a-apple-n/,AS USA,"La demanda acusa a Apple de mantener un monopolio en el mercado de smartphones al restringir el acceso a su hardware y software, lo que podría aumentar los costos para los consumidores y sofocar la innovación. Esta acción legal generó incertidumbre entre los inversores, provocando una caída en el precio de las acciones de la compañía."


## Export results

In [20]:
df.to_csv('outputs/news.xlsx', index=False)

In [21]:
df.to_csv('outputs/news.csv', index=False)